<h1>sum_emissions</h1>
<h3>Notebook for printing emissions totals from HEMCO diagnostic netCDF files</h3>

We only need to import a few packages. The os.path package is used for filename manipulation, xarray for manipulationg netCDF data, and numpy for math operations

In [27]:
import os
import xarray as xr
import numpy as np

Define the data file that you want to read.  We will sum emissions for each of the variables within this file.


In [28]:
# We will use the July 2016 data file as an example.
# You can edit this accordingly to use any file that you have on hand.
filename = "HEMCO_sa.diagnostics.201607010000.nc"

# Year and month 
year  = 2016
month = 7

# Seconds in July 2016
sec_in_month = 31.0 * 86400.0

We will read the entire contents of the netCDF file into an xarray Dataset object.  The very powerful xarray package allows you to easily read and manipulate Earth Science data that are stored in netCDF files.

In [29]:
# Load the netCDF file into an xarray dataset
ds = xr.open_dataset(filename)

# Show the contents of the xarray dataset object.
# In Jupyter notebook, typing the name of a variable or object
# on the last line of a cell prints the object.
ds

<xarray.Dataset>
Dimensions:      (lat: 46, lev: 47, lon: 72, time: 1)
Coordinates:
  * lon          (lon) float64 -180.0 -175.0 -170.0 -165.0 -160.0 -155.0 ...
  * lat          (lat) float64 -89.0 -86.0 -82.0 -78.0 -74.0 -70.0 -66.0 ...
  * lev          (lev) float64 0.9925 0.9775 0.9625 0.9475 0.9325 0.9175 ...
  * time         (time) datetime64[ns] 2016-07-01
Data variables:
    hyam         (lev) float64 2.402 332.1 986.4 1.637e+03 2.285e+03 ...
    hybm         (lev) float64 0.9925 0.9742 0.9526 0.9311 0.9096 0.8882 ...
    P0           float64 1e+05
    AREA         (lat, lon) float64 2.16e+09 2.16e+09 2.16e+09 2.16e+09 ...
    POG2_TOTAL   (time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    POG1_TOTAL   (time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    OCPO_TOTAL   (time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    OCPI_TOTAL   (time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    BCPO_TOTAL   (time, lat, lon) float64 0.0 0.0 

Extract the grid box surface area values from the HEMCO diagnostic file.  Surface area is stored in each HEMCO diagnostic file as a 2-dimensional (lon,lat) variable named "AREA".

In [30]:
# Extract the grid box surface area values f(in units of m^2) 
# from the xarray Dataset object into a numpy array.
# This is done by using with the .values tag.
area = ds['AREA'].values 

# Print the shape of the area array
print( "Original shape of AREA: {}".format( area.shape ) )

# Reshape area from (46,72) to (1,46,72) to be consistent with the 
# emissions data variables in the file.  If the array shapes are the same,
# we can multiply each emissions data array directly by the area array.
area = area[ np.newaxis, :, : ]

# Print the new of the area array
print( "New shape of AREA: {}".format( area.shape ) )

Original shape of AREA: (46, 72)
New shape of AREA: (1, 46, 72)


Construct the list of variables for which emissions totals will be printed.

In [32]:
# List of all variables in the xarray object.
# This will include the index arrays (lon, lat, time) and also the AREA variable.
varlist = ds.data_vars.keys()

# Restrict the list of variables to only those containing emissions data.
# (i.e. those that have at least 3 dimensions (e.g. time,lat.lon).
# To manipulate a list, we have to use the "v for v in varlist if ..." syntax.
varlist = [v for v in varlist if ds[v].ndim > 2  ]

# Print the modified list.  This now only contains emisisons variables.
varlist

['POG2_TOTAL',
 'POG1_TOTAL',
 'OCPO_TOTAL',
 'OCPI_TOTAL',
 'BCPO_TOTAL',
 'BCPI_TOTAL',
 'HCOOH_TOTAL',
 'XYLE_TOTAL',
 'TOLU_TOTAL',
 'BENZ_TOTAL',
 'C2H4_TOTAL',
 'C2H2_TOTAL',
 'GLYC_TOTAL',
 'HAC_TOTAL',
 'C2H6_TOTAL',
 'CH2O_TOTAL',
 'PRPE_TOTAL',
 'MVK_TOTAL',
 'ALD2_TOTAL',
 'MEK_TOTAL',
 'ISOP_TOTAL',
 'ALK4_TOTAL',
 'NH3_TOTAL',
 'pFe_TOTAL',
 'SO4_TOTAL',
 'SO2_TOTAL',
 'NO2_TOTAL',
 'NO_TOTAL',
 'SOAP_TOTAL',
 'CO_TOTAL']

Loop over each emissions variable in the list and print the emission totals.

In [33]:
for name in varlist:
    
    # Get the units of each data variable.
    # This will typically be in either kg/m2/s or kgC/m2/s, depending on the variable.
    units = ds[name].units
    
    # We are going to print emissions totals in kg, so let's strip off just the
    # kg or kgC part from the units name.  We do this the string "split()" method.
    units = units.split( "/" )
    
    # units.split will return an array of substrings.  We just want the first element.
    units = units[0]
    
    # Extract the data from the "ds" xarray object into a numpy array object.
    emissions_array = ds[name].values
    
    # Convert the emissions data to kg/m2/s and take the sum
    emissions_sum = np.sum( emissions_array * area * sec_in_month )
    
    # Print the emissions total. The  " .6e" specifier requests a number in 
    # scientific notation with 6 decimal places.  We leave a space for the - sign.
    # The string method ".ljust(x)" will left-justify a string in a column x spaces wide.
    print( "{n} sum = {es: .6e}  {u}".format( n=name.ljust(15), es=emissions_sum, u=units ) )

POG2_TOTAL      sum =  5.239687e+06  kgC
POG1_TOTAL      sum =  5.034209e+06  kgC
OCPO_TOTAL      sum =  8.857803e+07  kgC
OCPI_TOTAL      sum =  8.857803e+07  kgC
BCPO_TOTAL      sum =  4.223591e+07  kgC
BCPI_TOTAL      sum =  1.055898e+07  kgC
HCOOH_TOTAL     sum =  2.068979e+07  kg
XYLE_TOTAL      sum =  2.153014e+06  kgC
TOLU_TOTAL      sum =  6.596417e+07  kgC
BENZ_TOTAL      sum =  6.755955e+07  kgC
C2H4_TOTAL      sum =  7.328033e+07  kgC
C2H2_TOTAL      sum =  1.380832e+07  kgC
GLYC_TOTAL      sum =  1.315163e+07  kg
HAC_TOTAL       sum =  1.258072e+08  kg
C2H6_TOTAL      sum =  7.362613e+07  kgC
CH2O_TOTAL      sum =  6.660128e+07  kg
PRPE_TOTAL      sum =  3.889013e+07  kgC
MVK_TOTAL       sum =  5.276616e+06  kg
ALD2_TOTAL      sum =  1.840918e+07  kgC
MEK_TOTAL       sum =  8.520004e+06  kgC
ISOP_TOTAL      sum =  3.381371e+06  kgC
ALK4_TOTAL      sum =  1.940089e+08  kgC
NH3_TOTAL       sum =  3.007598e+08  kg
pFe_TOTAL       sum =  3.137024e+05  kg
SO4_TOTAL       sum =  